In [ ]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

In [ ]:
%load_ext nb_mypy

In [ ]:
Variable = str
Literal  = Variable | tuple[str, Variable]
Clause   = frozenset[Literal]

# The <a href="https://en.wikipedia.org/wiki/DPLL_algorithm">Davis-Putnam Algorithm</a> with the Jeroslow-Wang Heuristic

This notebook implements the algorithm of Davis and Putnam.  Further details about this algorithm are provided in the lecture notes.

The function `complement(l)` computes the complement of a literal `l`.
If $p$ is a propositional variable, we have the following: 
* $\texttt{complement}(p) = \neg p$,
* $\texttt{complement}(\neg p) = p$.

As we are working with clauses that result form transforming given formulas into *conjunctive normal form* and these clauses do not contain $\top$ or $\bot$, we don't have to bother with $\top$ or $\bot$ in this function.

In [ ]:
def complement(l: Literal) -> Literal:
    'Compute the complement of the literal L.'
    match l:
        case ('¬', p): return p
        case p       : return ('¬', p) # type: ignore

The function `extractVariable(l)` extracts the variable from the literal `l`.
If $p$ is a propositional variable, we have the following: 
* $\texttt{extractVariable}(p) = p$,
* $\texttt{extractVariable}(\neg p) = p$.

In [ ]:
def extractVariable(l: Literal) -> Variable:
    'Extract the propositional variable from the literal l.'
    match l:
        case ('¬', p): return p
        case p       : return p # type: ignore

The function `arb(S)` returns an arbitrary element from the set `S`.

In [ ]:
def arb[E](S: set[E] | frozenset[E]) -> E:
    'Return some member from the set S.'
    for x in S: 
        return x
    return None # type: ignore

The function `selectLiteral(Clauses, Forbidden)`
returns a literal from a clause from the set `Clauses` such that the variable of this literal does not occur in the set `Forbidden`.  It uses the *Jereslow-Wang heuristic* to choose the literal. The Jereslow-Wang heuristic $\texttt{JW}(l)$ of a literal $l$ in a set of clauses is defined as follows:
$$ \texttt{JW}(\textrm{Clauses}, l) = \sum\limits_{\{\,C \in \texttt{Clauses}\;\mid\; l \in C\;\}} \frac{1}{\;2^{|C|}\;} $$ 
Here, $|C|$ denotes the number of literals in the clause $C$.  The idea is to choose a literal that occurs in many clauses and therefore subsumes a lot of clauses.

In [ ]:
def selectLiteral(Clauses  : set[Clause], 
                  Variables: set[Variable], 
                  UsedVars : set[Variable]) -> Literal:
    Scores: dict[Literal, float] = {} 
    for var in Variables:
        if var not in UsedVars:
            cmp = ('¬', var)
            Scores[var] = 0.0
            Scores[cmp] = 0.0
            for C in Clauses:
                if cmp in C:
                    Scores[cmp] += 2 ** -len(C)
                if var in C: 
                    Scores[var] += 2 ** -len(C)
    return max(Scores, key=Scores.get) # type: ignore

In [ ]:
D = { 'a': 3, 'b': 5, 'c': 1}
max(D, key=D.get) # type: ignore

Given a set of clauses `Clauses` and a literal `l`, the procedure `reduce(Clauses, l)` performs all unit cuts and all unit subsumptions on clauses of of the set `Clauses` that are possible using the unit clause $\{\mathtt{l}\}$.  The resulting set of clauses is returned.  Mathematically, the function `reduce` is defined as follows:
$$\texttt{reduce}(\texttt{Clauses},l)  := 
 \Bigl\{\, C \backslash \bigl\{\overline{\,l\,}\bigr\} \;|\; C \in \texttt{Clauses} \wedge \overline{\,l\,} \in C \,\Bigr\} 
       \,\cup\, \Bigl\{\, C \in \texttt{Clauses} \mid \overline{\,l\,} \not\in C \wedge l \not\in C \Bigr\} \cup \bigl\{\{l\}\bigr\}.
$$
This function should only be called if the unit clause $\{l\}$ is an element of the set `Clauses`.

In [ ]:
def reduce(Clauses: set[Clause], l: Literal) -> set[Clause]:
    lBar = complement(l)  
    return   { C - { lBar } for C in Clauses if lBar     in C                }  \
           | { C            for C in Clauses if lBar not in C and l not in C }  \
           | { frozenset({l}) }

`Clauses` is a set of clauses.  The call `saturate(Clauses)` computes the set of those clauses that can be derived from `Clauses` via repeated applications of unit cuts or unit subsumptions.

In [ ]:
def saturate(Clauses: set[Clause]) -> set[Clause]:
    S     = Clauses.copy()
    Units = { C for C in S if len(C) == 1 } # set of unit clauses occurring in C
    Used  = set()                           # remember which unit clauses have already been used
    while len(Units) > 0:  # iterate as long as we derive new unit clauses
        unit  = Units.pop()
        Used |= { unit }
        l     = arb(unit)
        S     = reduce(S, l)
        Units = { C for C in S if len(C) == 1 } - Used        
    return S

The function `solve(Clauses)` takes a set of clauses  as input.  The function tries to compute a variable assignment that satisfies all clauses in `Clauses`.  If this is successful, a set of unit clauses is returned.  This set of unit clauses does not contain  any complementary literals and therefore corresponds to a variable assignment satisfying all clauses.  If the set `Clauses` is unsatisfiable, then the set `{{}}` is returned instead.

The real work is done by the function `solve_recursive`.  This function takes two additional arguments:
* `Variables` is the set of all variables occurring in `Clauses`.
* `UsedVars`  is the set of those variables that have already been used in case distinctions.

In [ ]:
def solve_recursive(Clauses: set[Clause], Variables: set[Variable], UsedVars: set[Variable]) -> set[Clause]:
    S  = saturate(Clauses)
    Empty: frozenset[Literal] = frozenset()
    Falsum = { Empty }
    if Empty in S:                  # S is inconsistent
        return Falsum           
    if all(len(C) == 1 for C in S): # S is trivial
        return S
    # use the Jereslow-Wang heuristic to select the most promising literal l
    l       = selectLiteral(S, Variables, UsedVars)
    lBar    = complement(l)
    p       = extractVariable(l)
    newUsed = UsedVars | { p }
    Result  = solve_recursive(S | { frozenset({l}) }, Variables, newUsed)
    if Result != Falsum:
        return Result
    return solve_recursive(S | { frozenset({lBar}) }, Variables, newUsed)

In [ ]:
def solve(Clauses: set[Clause]) -> set[Clause]:
    Variables = { extractVariable(l) for C in Clauses for l in C }
    return solve_recursive(Clauses, Variables, set())

The function $\texttt{toString}(S)$ takes a set $S$ as input.  The set $S$ is a set of frozensets and the function converts $S$ into a string that looks like a set of sets.  This is only used for pretty printing.

In [ ]:
def literal_to_str(C: Clause) -> str:
    'Convert a unit clause to a string.'
    l = arb(C)
    if l[0] == '¬':
        return f'{str(l[1])} ↦ False' 
    else:
        return f'{str(l)} ↦ True'
    
def toString(S: set[Clause], Simplified: set[Clause]) -> str:
    'Convert the set S of frozen sets to a string where frozen sets are written as sets.'
    if len(Simplified) == 1:
        Clause = arb(Simplified)
        if len(Clause) == 0:
            return f'{prettify(S)} is unsolvable'
    else:
        result = '{ ' + ', '.join({ literal_to_str(C) for C in Simplified }) + ' }'
        return result
    return None # type: ignore

def prettify(Clauses: set[Clause]) -> str:
    return ', '.join({str(set(C)) for C in Clauses})

In [ ]:
c1: Clause = frozenset({ 'r', 'p', 's' })
c2: Clause = frozenset({ 'r', 's' })
c3: Clause = frozenset({ 'p', 'q', 's' })
c4: Clause = frozenset({ ('¬', 'p'), ('¬', 'q') })
c5: Clause = frozenset({ ('¬', 'p'), 's', ('¬', 'r') })
c6: Clause = frozenset({ 'p', ('¬', 'q'), 'r'})
c7: Clause = frozenset({ ('¬', 'r'), ('¬', 's'), 'q' })
c8: Clause = frozenset({ ('¬', 'p'), ('¬', 's')})
c9: Clause = frozenset({ 'p', ('¬', 'r'), ('¬', 'q') })
c0: Clause = frozenset({ ('¬', 'p'), 'r', 'q', ('¬', 's') })
S  = { c0, c1, c2, c3, c4, c5, c6, c7, c8, c9 }
print(toString(S, solve(S)))

In [ ]:
c11: Clause = frozenset({ 'p', 'r', 'q', ('¬', 's') })
S   = { c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c11 }
print(toString(S, solve(S)))

In [ ]:
c1: Clause = frozenset({ 'r', 'p', 's' })
c2: Clause = frozenset({ 'r', 's' })
c3: Clause = frozenset({ 'q', 'p', 's' })
c4: Clause = frozenset({ ('¬', 'p'), ('¬', 'q') })
c5: Clause = frozenset({ ('¬', 'p'), 's', ('¬', 'r') })
c6: Clause = frozenset({ 'p', ('¬', 'q'), 'r' })
c7: Clause = frozenset({ ('¬', 'r'), ('¬', 's'), 'q' })
c8: Clause = frozenset({ 'p', 'q', 'r', 's' })
c9: Clause = frozenset({ 'r', ('¬', 's'), 'q' })
c10: Clause = frozenset({ 's', ('¬', 'r'), ('¬', 'q') })
c11: Clause = frozenset({ 's', ('¬', 'r') })
c12: Clause = frozenset({ 'r', ('¬', 's') })
S  = {c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12}
solve(S)